In [2]:
import pandas as pd
import numpy as np

In [3]:
df_ast_r = pd.read_csv("/Users/taliyas/PycharmProjects/FinalProject/new df/df_ast_r.csv")
df_ast_s = pd.read_csv("/Users/taliyas/PycharmProjects/FinalProject/new df/df_ast_s.csv")
df_gens = pd.read_csv("/Users/taliyas/PycharmProjects/FinalProject/new df/df_gens.csv")
df_index = pd.read_csv("/Users/taliyas/PycharmProjects/FinalProject/new df/df_index.csv")
df_origin = pd.read_csv("/Users/taliyas/PycharmProjects/FinalProject/isolates (1).csv")
df_gens_anti_R = pd.read_csv('df_gens_anti_R.csv')
df_antibiotic_family = pd.read_csv('antibiotic_family.csv')

In [4]:
# First, merge the "aro-term" and "amr_gene_family" from df_gens_anti_R into the final_table.
final_table = df_gens_anti_R[['amr_gene_family', 'aro_term']].drop_duplicates().copy()
final_table.columns = ['gene family', 'gene']
final_table.shape

(1918, 2)

In [5]:
print(final_table)

                                               gene family  \
0                                                 ANT(2'')   
1011     major facilitator superfamily (MFS) antibiotic...   
15309                            sulfonamide resistant sul   
29319    resistance-nodulation-cell division (RND) anti...   
29320                                   OXA beta-lactamase   
...                                                    ...   
2920424         glycopeptide resistance gene cluster; vanR   
2920520                       fosfomycin thiol transferase   
2920522                                CARB beta-lactamase   
2920529  23S rRNA with mutation conferring resistance t...   
2920542            Cfr 23S ribosomal RNA methyltransferase   

                                                      gene  
0                                              ANT(2'')-Ia  
1011                                            qacEdelta1  
15309                                                 sul1  
29319      

In [6]:
# Convert pandas series to sets
set_gen = set(df_gens["gen"])
set_aro_term = set(df_gens_anti_R['aro_term'])

# Find the intersection (overlapping values) and count the number of items
overlap = set_gen.intersection(set_aro_term)
overlap_count = len(overlap)

print(overlap_count)


114


In [7]:
# Find the difference between set_gen and set_aro_term
genes_in_gen_not_in_aro_term = set_gen.difference(set_aro_term)
# Create a new dataframe for genes_in_gen_not_in_aro_term
genes_to_append = pd.DataFrame(genes_in_gen_not_in_aro_term, columns=['gene'])
genes_to_append['gene family'] = np.nan

# Append this dataframe to final_table
final_table = pd.concat([final_table, genes_to_append], axis=0)


In [8]:
# Add a dummy key to both dataframes
final_table['key'] = 1
df_antibiotic_family['key'] = 1

# Merge on the dummy key
final_table = pd.merge(final_table, df_antibiotic_family[['antibiotics', 'family', 'key']], on='key')

# Drop the dummy key
final_table.drop('key', axis=1, inplace=True)


In [9]:
# Drop duplicates based on 'gene' and 'antibiotic'

final_table.columns

Index(['gene family', 'gene', 'antibiotics', 'family'], dtype='object')

In [10]:
# Drop duplicates based on 'gene' and 'antibiotic'
final_table = final_table.drop_duplicates(subset=['gene', 'antibiotics'])


In [11]:
# Rename 'antibiotics' to 'antibiotic' and 'family' to 'drug_class'
final_table = final_table.rename(columns={'antibiotics': 'antibiotic', 'family': 'drug_class'})


In [12]:
final_table.shape

(264384, 4)

In [13]:
final_table

,gene family,gene,antibiotic,drug_class
0,ANT(2''),ANT(2'')-Ia,spectinomycin,aminoglycoside
1,ANT(2''),ANT(2'')-Ia,trimethoprim-sulfamethoxazole,sulfonamide
2,ANT(2''),ANT(2'')-Ia,azithromycin,macrolide
3,ANT(2''),ANT(2'')-Ia,ceftazidime-avibactam,beta-lactamase
4,ANT(2''),ANT(2'')-Ia,piperacillin,Penicillin
...,...,...,...,...
264379,NaN,blaOXY-5-4,ceftazidime,beta-lactamase
264380,NaN,blaOXY-5-4,amoxicillin-clavulanic acid,beta-lactamase
264381,NaN,blaOXY-5-4,streptomycin,Aminoglycoside
264382,NaN,blaOXY-5-4,moxifloxacin,Quinolone


In [14]:
final_table['gene'].nunique()

2754

In [15]:
final_table['label'] = None

In [16]:
final_table.columns

Index(['gene family', 'gene', 'antibiotic', 'drug_class', 'label'], dtype='object')

In [18]:
# Iterating over each index, antibiotic in df_ast_s
for _, row in df_ast_s.iterrows():
    index = row['index']
    antibiotic = row['antibiotic']

    # Finding corresponding gens in df_gens
    matching_gens = df_gens[df_gens['index'] == index]['gen']

    # Updating values in new_tab
    for gen in matching_gens:
       final_table.loc[(final_table['gene'] == gen) & (final_table['antibiotic'] == antibiotic), 'label'] = 'S'
final_table.to_csv("Final_Table.csv", index=False)

In [19]:
final_table

,gene family,gene,antibiotic,drug_class,label
0,ANT(2''),ANT(2'')-Ia,spectinomycin,aminoglycoside,None
1,ANT(2''),ANT(2'')-Ia,trimethoprim-sulfamethoxazole,sulfonamide,None
2,ANT(2''),ANT(2'')-Ia,azithromycin,macrolide,None
3,ANT(2''),ANT(2'')-Ia,ceftazidime-avibactam,beta-lactamase,None
4,ANT(2''),ANT(2'')-Ia,piperacillin,Penicillin,None
...,...,...,...,...,...
264379,NaN,blaOXY-5-4,ceftazidime,beta-lactamase,None
264380,NaN,blaOXY-5-4,amoxicillin-clavulanic acid,beta-lactamase,None
264381,NaN,blaOXY-5-4,streptomycin,Aminoglycoside,None
264382,NaN,blaOXY-5-4,moxifloxacin,Quinolone,None


In [20]:
final_table.to_csv("Final_Table_last.csv", index=False)

In [21]:
final_table['gene'].nunique()

2754

In [22]:
final_table['antibiotic'].nunique()

96